<a href="https://colab.research.google.com/github/Ruwai/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/module2-join-datasets/LS_DS_122_Join_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-01-16 00:03:47--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.226.155
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.226.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.5’

instacart_online_gr 100%[===================>] 196.03M  6.74MB/s    in 14s     

2019-01-16 00:04:01 (14.0 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.5’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


## Goal: Reproduce this example

The first two orders for user id 1:

In [4]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [5]:
!ls -lh

total 877M
-rw-r--r-- 1  502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1  502 staff  270 May  2  2017 departments.csv
drwxr-xr-x 2  502 staff 4.0K May  2  2017 instacart_2017_05_01
-rw-r--r-- 1 root root  197M Oct 20  2017 instacart_online_grocery_shopping_2017_05_01.tar.gz
-rw-r--r-- 1  502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1  502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1  502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1  502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

### aisles

In [0]:
import pandas as pd
aisles = pd.read_csv('aisles.csv')

In [7]:
aisles.shape

(134, 2)

In [8]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [9]:
display(example)

### departments

In [0]:
departments = pd.read_csv('departments.csv')

In [11]:
departments.shape

(21, 2)

In [12]:
departments.sample(n=5)

,department_id,department
20,21,missing
14,15,canned goods
10,11,personal care
9,10,bulk
3,4,produce


In [13]:
display(example)

### order_products__prior

In [0]:
order_products__prior = pd.read_csv('order_products__prior.csv')

In [15]:
order_products__prior.shape

(32434489, 4)

In [16]:
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### order_products__train

In [0]:
order_products__train = pd.read_csv('order_products__train.csv')

In [18]:
order_products__train.shape

(1384617, 4)

In [19]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


### orders

In [0]:
orders = pd.read_csv('orders.csv')

In [21]:
orders.shape

(3421083, 7)

In [22]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### products

In [0]:
products = pd.read_csv('products.csv')

In [24]:
products.shape

(49688, 4)

In [25]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## Concatenate order_products__prior and order_products__train

In [0]:
order_products = pd.concat([order_products__prior, order_products__train])

In [27]:
order_products.shape, order_products__prior.shape, order_products__train.shape

((33819106, 4), (32434489, 4), (1384617, 4))

In [0]:
assert len(order_products) == len(order_products__prior) + len(order_products__train)

In [0]:
assert(len(order_products.columns) ==
      len(order_products__prior.columns) ==
      len(order_products__train.columns))



## Get a subset of orders — the first two orders for user id 1

In [0]:
condition = (orders.user_id==1) & (orders.order_number <= 2)

columns =['user_id',
          'order_id',
          'order_number',
          'order_dow',
          'order_hour_of_day']

subset = orders.loc[condition,columns]

## Merge dataframes

In [0]:
columns = ['order_id', 'add_to_cart_order', 'product_id']
merged = pd.merge(subset, order_products[columns],
                 how='inner', on='order_id')


In [32]:
subset.shape, merged.shape

((2, 5), (11, 7))

In [33]:
subset

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


In [34]:
merged

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id
0,1,2539329,1,2,8,1,196
1,1,2539329,1,2,8,2,14084
2,1,2539329,1,2,8,3,12427
3,1,2539329,1,2,8,4,26088
4,1,2539329,1,2,8,5,26405
5,1,2398795,2,3,7,1,196
6,1,2398795,2,3,7,2,10258
7,1,2398795,2,3,7,3,12427
8,1,2398795,2,3,7,4,13176
9,1,2398795,2,3,7,5,26088


In [35]:
display(example)

In [0]:
final = pd.merge(merged, products[['product_id', 'product_name']])
final = final.sort_values(by=['order_number', 'add_to_cart_order'])

In [37]:
display(example)

In [0]:
final.columns = [column.replace('_', ' ')
                for column in final]

In [39]:
final.head()

,user id,order id,order number,order dow,order hour of day,add to cart order,product id,product name
0,1,2539329,1,2,8,1,196,Soda
2,1,2539329,1,2,8,2,14084,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,3,12427,Original Beef Jerky
5,1,2539329,1,2,8,4,26088,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,5,26405,XL Pick-A-Size Paper Towel Rolls


# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.

In [0]:
# c1 = ['order_id', 'order_number']
# c2 = ['product_id', 'product_name']

# sub = orders.loc[c1]
# sub

## i'm confused

In [41]:
my_products = ('Banana', 
               'Bag of Organic Bananas', 
               'Organic Strawberries', 
               'Organic Baby Spinach',
               'Organic Hass Avocado',
               'Large Lemon', 
               'Strawberries',
               'Limes',
               'Organic Whole Milk')

my_products


('Banana',
 'Bag of Organic Bananas',
 'Organic Strawberries',
 'Organic Baby Spinach',
 'Organic Hass Avocado',
 'Large Lemon',
 'Strawberries',
 'Limes',
 'Organic Whole Milk')

In [0]:
# condition = orders.
# cols = ['order_id',
#         'order_number']

# sub = orders.loc[orders,cols]

# sub

In [43]:
ex_merge = pd.merge(order_products, orders)
ex_merge.head()
#drop 

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [44]:
#dropping stuff 
ex_merge = ex_merge.drop(['order_id', 'user_id', 'eval_set',
                          'order_dow', 'order_hour_of_day',
                          'days_since_prior_order'], axis=1)
ex_merge.head()

,product_id,add_to_cart_order,reordered,order_number
0,33120,1,1,3
1,28985,2,1,3
2,9327,3,0,3
3,45918,4,1,3
4,30035,5,0,3


In [45]:
final_hopefully = pd.merge(ex_merge, products[['product_id','product_name']],
                          how='inner', on='product_id')

final_hopefully.tail()

,product_id,add_to_cart_order,reordered,order_number,product_name
33819101,27499,31,0,27,Non-Dairy Coconut Seven Layer Bar
33819102,27499,9,0,30,Non-Dairy Coconut Seven Layer Bar
33819103,37703,15,0,5,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...
33819104,43725,25,0,8,Sweetart Jelly Beans
33819105,36233,6,0,34,Water With Electrolytes


In [46]:
final_hopefuly = final_hopefully.sort_values(by=['product_name'])

final_hopefully

,product_id,add_to_cart_order,reordered,order_number,product_name
0,33120,1,1,3,Organic Egg Whites
1,33120,5,0,2,Organic Egg Whites
2,33120,13,0,11,Organic Egg Whites
3,33120,5,1,21,Organic Egg Whites
4,33120,28,1,48,Organic Egg Whites
5,33120,2,1,15,Organic Egg Whites
6,33120,7,1,6,Organic Egg Whites
7,33120,5,1,11,Organic Egg Whites
8,33120,1,1,63,Organic Egg Whites
9,33120,4,1,16,Organic Egg Whites


In [0]:
### Make index = my_products

In [48]:
new_df = final_hopefully.copy()
new_df.head()

,product_id,add_to_cart_order,reordered,order_number,product_name
0,33120,1,1,3,Organic Egg Whites
1,33120,5,0,2,Organic Egg Whites
2,33120,13,0,11,Organic Egg Whites
3,33120,5,1,21,Organic Egg Whites
4,33120,28,1,48,Organic Egg Whites


In [49]:
new_df = new_df.set_index(['product_name'])
new_df.head()

,product_id,add_to_cart_order,reordered,order_number
product_name,,,,
Organic Egg Whites,33120,1,1,3
Organic Egg Whites,33120,5,0,2
Organic Egg Whites,33120,13,0,11
Organic Egg Whites,33120,5,1,21
Organic Egg Whites,33120,28,1,48


In [0]:
new_df = new_df.loc[new_df.index.isin(['Banana', 
               'Bag of Organic Bananas', 
               'Organic Strawberries', 
               'Organic Baby Spinach',
               'Organic Hass Avocado',
               'Organic Avocado',
               'Large Lemon', 
               'Strawberries',
               'Limes',
               'Organic Whole Milk']
)]
##
# Banana
# Bag of Organic Bananas
# Organic Strawberries
# Organic Baby Spinach
# Organic Hass Avocado
# Organic Avocado
# Large Lemon
# Strawberries
# Limes
# Organic Whole Milk

In [0]:
# ### set
# map(new_df['product_id'].value_counts(), new_df)
# new_df['product_count'] = new_df['product_id'].apply()
# new_df[new_df['product_count']]

In [0]:
# new_df['product_count'] = [sum(new_df['product_id'] == new_df['product_id'][i])
#                           for i in range(len(new_df))]
# new_df

In [0]:
counts = new_df['product_id'].value_counts()

counts_col = counts.to_dict()

new_df['counts'] = new_df['product_id'].map(counts_col)

In [54]:
new_df

,product_id,add_to_cart_order,reordered,order_number,counts
product_name,,,,,
Organic Baby Spinach,21903,4,1,16,251705
Organic Baby Spinach,21903,6,0,2,251705
Organic Baby Spinach,21903,3,1,7,251705
Organic Baby Spinach,21903,4,0,8,251705
Organic Baby Spinach,21903,8,1,8,251705
Organic Baby Spinach,21903,1,0,1,251705
Organic Baby Spinach,21903,4,1,13,251705
Organic Baby Spinach,21903,2,1,13,251705
Organic Baby Spinach,21903,4,0,2,251705
